In [ ]:
#| default_exp config

## Configuration

### Imports

In [ ]:
#| export
from dataclasses import dataclass
from fastcore.all import *
from fastcore.xdg import *
from typing import get_type_hints

In [ ]:
#| export
_tui_writer_home_dir = 'tui_writer'
_tui_writer_cfg_name = 'tui_writer.conf'

In [ ]:
#| export
def _cfg_path(): return xdg_config_home() / _tui_writer_home_dir / _tui_writer_cfg_name

In [ ]:
_cfg_path()

In [ ]:
#| export
@dataclass
class TuiWriterConfig:
    openai_key: str = ''
    anthropic_key: str = ''
    gemini_key: str = ''
    last_provider: str = ''
    last_model: str = ''

In [ ]:
cfg = TuiWriterConfig()
cfg

In [ ]:
#| export
def get_cfg():
    path = _cfg_path()
    path.parent.mkdir(parents=True, exist_ok=True)
    _types = get_type_hints(TuiWriterConfig)
    return Config(
        path.parent,
        path.name,
        create=asdict(TuiWriterConfig()),
        types=_types,
        inline_comment_prefixes=('#')
    )